# Imports

In [1]:
from setup import *
from genetic_algorithm import create_roadmap, calculate_fitness, crossover, mutate

import random
import pandas as pd
from itertools import product

# Parametros

In [2]:
population_sizes = [3, 5, 8]
n_generations_list = [100, 150, 200]
crossover_methods = [CrossoverMethod.OX, CrossoverMethod.CX, CrossoverMethod.OX1]
mutation_methods = [MutateMethod.SWAP, MutateMethod.INSERTION, MutateMethod.INVERSION, MutateMethod.SHUFFLE]
mutation_probabilities = [0.05, 0.1, 0.2]

# Funções

In [3]:
def get_winner_genetic_algorithm(
        population_size: int,
        n_generations: int,
        crossover_method: CrossoverMethod,
        mutation_method: MutateMethod, 
        mutation_probability: float) -> dict:
    winner = {
        'generation': None,
        'best_fitness': None,
        'solution': None
    }
    generation = 0
    population = [create_roadmap(ATTRACTIONS) for _ in range(population_size)] 
    best_fitness_values = []
    best_solutions = []
    for generation in range(n_generations):
        population.sort(key=lambda it: calculate_fitness(it, BUDGET_MAX), reverse=False)

        new_population = [population[0]]

        best_fitness = calculate_fitness(population[0], BUDGET_MAX)
        best_solution = population[0]

        if(best_solution != winner["solution"]):
            winner['generation'] = generation
            winner['best_fitness'] = best_fitness
            winner['solution'] = best_solution

        best_fitness_values.append(best_fitness)
        best_solutions.append(best_solution)

        generation += 1

        while len(new_population) < population_size:
            parent1, parent2 = random.choices(population[:10], k=2)

            child1, child2 = crossover(crossover_method, parent1, parent2)

            child1 = mutate(mutation_method, child1, mutation_probability)
            child2 = mutate(mutation_method, child2, mutation_probability)

            new_population.append(child1)
            new_population.append(child2)

        population = new_population
    
    return winner

def grid_search_genetic_algorithm():
    param_grid = list(product(population_sizes, n_generations_list, crossover_methods, mutation_methods, mutation_probabilities))
    results = []

    for population_size, n_generations, crossover_method, mutation_method, mutation_probability in param_grid:
        result = get_winner_genetic_algorithm(
            population_size=population_size,
            n_generations=n_generations,
            crossover_method=crossover_method,
            mutation_method=mutation_method,
            mutation_probability=mutation_probability
        )

        results.append({
            "population_size": population_size,
            "n_generations": n_generations,
            "crossover_method": crossover_method,
            "mutation_method": mutation_method,
            "mutation_probability": mutation_probability,
            "generation": result["generation"],
            "best_fitness": result["best_fitness"]
        })
    
    return results

In [4]:
results = grid_search_genetic_algorithm()

In [5]:
df = pd.DataFrame(results)

In [6]:
df

,population_size,n_generations,crossover_method,mutation_method,mutation_probability,generation,best_fitness
0,3,100,CrossoverMethod.OX,MutateMethod.SWAP,0.05,35,-13.074414
1,3,100,CrossoverMethod.OX,MutateMethod.SWAP,0.10,17,-11.942777
2,3,100,CrossoverMethod.OX,MutateMethod.SWAP,0.20,46,-11.243443
3,3,100,CrossoverMethod.OX,MutateMethod.INSERTION,0.05,13,-11.613164
4,3,100,CrossoverMethod.OX,MutateMethod.INSERTION,0.10,0,-11.942777
...,...,...,...,...,...,...,...
319,15,200,CrossoverMethod.OX1,MutateMethod.INVERSION,0.10,11,-13.121667
320,15,200,CrossoverMethod.OX1,MutateMethod.INVERSION,0.20,19,-13.121667
321,15,200,CrossoverMethod.OX1,MutateMethod.SHUFFLE,0.05,164,-13.074706
322,15,200,CrossoverMethod.OX1,MutateMethod.SHUFFLE,0.10,4,-13.121667


In [9]:
df.sort_values(['best_fitness', 'generation'], ascending=[True, True])

,population_size,n_generations,crossover_method,mutation_method,mutation_probability,generation,best_fitness
163,8,150,CrossoverMethod.CX,MutateMethod.INVERSION,0.10,0,-13.121667
274,15,150,CrossoverMethod.CX,MutateMethod.SHUFFLE,0.10,0,-13.121667
222,15,100,CrossoverMethod.OX,MutateMethod.INVERSION,0.05,1,-13.121667
290,15,200,CrossoverMethod.OX,MutateMethod.SWAP,0.20,1,-13.121667
296,15,200,CrossoverMethod.OX,MutateMethod.INVERSION,0.20,2,-13.121667
...,...,...,...,...,...,...,...
33,3,100,CrossoverMethod.OX1,MutateMethod.SHUFFLE,0.05,2,-7.583845
57,3,150,CrossoverMethod.CX,MutateMethod.SHUFFLE,0.05,61,-7.196232
105,3,200,CrossoverMethod.OX1,MutateMethod.SHUFFLE,0.05,1,-6.934905
16,3,100,CrossoverMethod.CX,MutateMethod.INSERTION,0.10,83,-6.878282


In [ ]:
df.